In [4]:
# A notebook to test interacting with Google Calendar API
# Initial imports
import os
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import os
import streamlit as st
from googleapiclient.discovery import build
from google.oauth2 import service_account
from dotenv import load_dotenv
from dateutil.parser import parse


In [35]:
credentials = service_account.Credentials.from_service_account_info(st.secrets["gcp_service_account"])
scoped_credentials = credentials.with_scopes(scopes=['https://www.googleapis.com/auth/calendar'])

In [36]:
credentials.token

In [42]:
# Build the service object for interacting with the Google Calendar API
service = build('calendar', 'v3', credentials=credentials)
service

In [43]:
# Query the API for the busy / free times of a given calendar
# The calendar ID can be found in the settings of the calendar
# The timeMin and timeMax parameters are optional

calendar_id = 'sjufan84@gmail.com'
time_min = '2021-01-01T00:00:00Z'
time_max = '2021-01-31T00:00:00Z'

# Call the API and store the results in a variable called events_result
events_result = service.events().list(calendarId=calendar_id, timeMin=time_min, timeMax=time_max, singleEvents=True, orderBy='startTime').execute()

In [55]:
import pytz
from datetime import datetime, timedelta
from googleapiclient.errors import HttpError

def find_available_time_slots(service, calendar_id, duration_minutes, start_time, end_time, timezone='UTC'):
    now = datetime.now(pytz.timezone(timezone))
    start_date = now.date()
    end_date = start_date + timedelta(days=7)

    start_time = datetime.strptime(start_time, '%H:%M').time()
    end_time = datetime.strptime(end_time, '%H:%M').time()

    freebusy_request = {
        "timeMin": now.isoformat(),
        "timeMax": (now + timedelta(days=7)).isoformat(),
        "timeZone": timezone,
        "items": [{"id": calendar_id}]
    }

    try:
        freebusy_response = service.freebusy().query(body=freebusy_request).execute()

        busy_periods = freebusy_response['calendars'][calendar_id]['busy']

        available_slots = []
        for i in range(7):
            day_start = datetime.combine(start_date + timedelta(days=i), start_time, pytz.timezone(timezone))
            day_end = datetime.combine(start_date + timedelta(days=i), end_time, pytz.timezone(timezone))
            
            prev_busy_end = day_start

            for busy_period in busy_periods:
                busy_start = datetime.fromisoformat(busy_period['start']).astimezone(pytz.timezone(timezone))
                busy_end = datetime.fromisoformat(busy_period['end']).astimezone(pytz.timezone(timezone))

                if busy_start.date() == day_start.date():
                    free_slot_start = prev_busy_end
                    free_slot_end = busy_start

                    if (free_slot_end - free_slot_start).total_seconds() >= duration_minutes * 60:
                        available_slots.append({
                            "start": free_slot_start,
                            "end": free_slot_end
                        })

                    prev_busy_end = busy_end
            
            if (day_end - prev_busy_end).total_seconds() >= duration_minutes * 60:
                available_slots.append({
                    "start": prev_busy_end,
                    "end": day_end
                })

        return available_slots
    except HttpError as error:
        print(f"An error occurred: {error}")
        return None




In [56]:
calendar_id = 'sjufan84@gmail.com'
duration_minutes = 60
start_time = '09:00'
end_time = '17:00'
timezone = 'America/Los_Angeles'  # Change this to the desired timezone
service = service

if service:
    available_slots = find_available_time_slots(service, calendar_id, duration_minutes, start_time, end_time, timezone)
    print("Available time slots:")
    for slot in available_slots:
        print(f"{slot['start']} - {slot['end']}")



Available time slots:
2023-05-09 09:00:00-07:53 - 2023-05-09 20:53:42-07:00
2023-05-10 09:15:00-07:00 - 2023-05-10 12:00:00-07:00
2023-05-10 13:00:00-07:00 - 2023-05-10 17:30:00-07:00
2023-05-10 18:30:00-07:00 - 2023-05-10 19:30:00-07:00
2023-05-11 09:15:00-07:00 - 2023-05-11 11:00:00-07:00
2023-05-11 13:00:00-07:00 - 2023-05-11 17:30:00-07:00
2023-05-11 18:30:00-07:00 - 2023-05-11 19:30:00-07:00
2023-05-12 09:15:00-07:00 - 2023-05-12 17:30:00-07:00
2023-05-12 18:30:00-07:00 - 2023-05-12 19:30:00-07:00
2023-05-13 09:15:00-07:00 - 2023-05-13 17:30:00-07:00
2023-05-13 18:30:00-07:00 - 2023-05-13 19:30:00-07:00
2023-05-14 09:15:00-07:00 - 2023-05-14 17:30:00-07:00
2023-05-14 18:30:00-07:00 - 2023-05-14 19:30:00-07:00
2023-05-15 09:15:00-07:00 - 2023-05-15 17:30:00-07:00
2023-05-15 18:30:00-07:00 - 2023-05-15 19:30:00-07:00
